# Keys 1981 Cubic Convolution

This notebook aims to experiment with the "Cubic Convolution" interpolation
algorithm provided by Keys in 1981 (IEEE Journal of something).


In [ ]:
import PIL.Image as Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
rec601_luma_weights = {
    # source:
    # https://en.wikipedia.org/wiki/Grayscale#Luma_coding_in_video_systems
    "r": 0.299,
    "g": 0.587,
    "b": 0.114
}
# load image data
img = Image.open('../media/mgsv.png')

# convert to grayscale and resize to 600x600
data = np.array(img)[480:1080, 600:1200]

snake = np.uint8(
    rec601_luma_weights['r'] * data[:, :, 0] + \
    rec601_luma_weights['g'] * data[:, :, 1] + \
    rec601_luma_weights['b'] * data[:, :, 2])

width, height = snake.shape


In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(snake, cmap='gray');

In [ ]:
def u(s: float):
    """
    cubic convolution interpolation kernel
    Keys 1981 "Cubic Convolution Interpolation for Digital Image Processing"

    :param s: kernel input value
    :return: kernel value for `s`
    """
    s = abs(s)
    # use <= instead of < to avoid None on s==0 or s==1 or s==2
    if 0.0 <= s < 1.0:
        return 1.5 * (s ** 3) - 2.5 * (s ** 2) + 1.0
    if 1.0 <= s < 2.0:
        return -0.5 * (s ** 3) + 2.5 * (s ** 2) - 4 * s + 2.0
    if 2.0 <= s:
        return 0.0

In [ ]:
x = np.linspace(-3, 3, 10_000)
y = np.array([u(a) for a in x])

plt.figure(figsize=(8, 5))
plt.title("Keys (1981) Cubic Convolution Kernel")
plt.grid()
plt.plot(x, y, color="steelblue");

In [ ]:
target_width, target_height = 1000, 1000
target_resolution = target_width, target_height
target_resolution

In [ ]:
first_line = snake[0]

plt.figure(figsize=(10, 10))
plt.imshow(snake, cmap="gray")
plt.plot(np.arange(600), first_line, color="orange");

In [ ]:
# == pixel centers ==
# sampling should be done at the "center" of each pixel.
#
# Assuming an image shall have the same actual target dimensions (in meters),
# and the coordinates of the image go from 0 to 1 (left to right/top to bottom),
# we want an easy formula to retrieve the center of a pixel s.t. we can sample
# from this with the correct weights to different pixel widths/heights.

from fractions import Fraction

orig_pixel_centers = [
    Fraction(n, width) + Fraction(1, 2) * Fraction(1, width)
    for n in range(width)
]

target_pixel_centers = [
    Fraction(n, target_width) + Fraction(1, 2) * Fraction(1, target_width)
    for n in range(target_width)
]

plt.figure(figsize=(20, 3))
plt.grid()

plt.scatter(orig_pixel_centers[:50], np.zeros(50),
            s=7, color="steelblue", label="Orig pixel centers")
plt.scatter(target_pixel_centers[:int(50 * target_width / width)], np.zeros(int(50 * target_width / width)),
            s=7, color="orange", label="Target Pixel centers")

plt.legend();
